# Proyecto Final 
## Curso de Teoría de la Simulación
### Impartido por ing. Uayeb Caballero

Proyecto desarrollado por: 
- Jeimie Vanesa Herrera
- Kattherine Mayely Hernandez

Consistente en la simulación del funcionamiento de un banco, donde el cliente llega con la idea del tipo de servicio que desea recibir, este puede ser:
- ATM (Cajero automático)
- Servicio personalizado: Caja o Servicio al cliente

### 1. Importamos librerías

In [157]:
import pandas as pd
import simpy
import random

### 2. Definimos la clase Banco que representará los recursos disponibles para los clientes (ATM, cajas y agentes de servicio al cliente)

In [158]:
class Banco:
    def __init__(self, env, num_atms, num_cajas, num_agentes):
        self.env = env
        self.atms = simpy.Resource(env, capacity=num_atms)
        self.cajas = simpy.Resource(env, capacity=num_cajas)
        self.servicio_cliente = simpy.Resource(env, capacity=num_agentes)

Definimos la estructura de los tipos de servicio y sus operaciones.

In [159]:
SERVICIOS = {
    "ATM": ["Deposito", "Retiro", "Consulta"],
    "caja": ["Deposito", "Retiro", "Consulta"],
    "servicio_cliente": ["Consulta", "Apertura de cuenta", "Problemas de cuenta"]
}


### 3. Definimos la clase que representa a un cliente en el banco

In [160]:
class Cliente:
    def __init__(self, env, id, banco):
        self.env = env
        self.id = id
        self.banco = banco
        self.servicio_personalizado = random.choice([True, False])
        self.tipo_servicio = None
        self.operacion = None
        self.eventos = pd.DataFrame()

        self._definir_servicio()

        self.env.process(self.start())

    # Definir el servicio que el cliente desea y la operación a realizar
    # Si el cliente no desea un servicio personalizado, se asigna "ATM" como servicio
    # y se elige una operación al azar de las disponibles para ese servicio
    def _definir_servicio(self):
        if self.servicio_personalizado:
            self.tipo_servicio = random.choice(list(SERVICIOS.keys()- ["ATM"]) )
        else:
            self.tipo_servicio = "ATM"
            
        self.operacion = random.choice(SERVICIOS[self.tipo_servicio])

    # Registrar eventos en un DataFrame
    # Cada evento tiene un ID, el tipo de evento, el tiempo en que ocurrió,
    def registrar_evento(self, evento):
        self.eventos = pd.concat([self.eventos, pd.DataFrame({
            "id": [self.id],
            "evento": [evento],
            "time": [self.env.now],
            "servicio": [self.tipo_servicio],
            "operacion": [self.operacion]
        })])

    # Iniciar el proceso del cliente
    def start(self):
        yield self.env.timeout( 3 )  # Esperar un tiempo antes de llegar al banco
        
        self.registrar_evento("LLEGÓ")

        # Asignar el recurso correspondiente según el tipo de servicio
        # y la operación a realizar
        if self.tipo_servicio == "ATM":
            recurso = self.banco.atms
            t_min, t_max = 1, 3 # Tiempo de espera en el ATM
        elif self.tipo_servicio == "caja":
            recurso = self.banco.cajas
            t_min, t_max = 3, 6 # Tiempo de esera 
        else:
            recurso = self.banco.servicio_cliente
            t_min, t_max = 15, 20

        # Se solicita el recurso correspondiente
        with recurso.request() as req:
            yield req # pide el recurso y se pone en espera si no está disponible
            self.registrar_evento("ATENDIDO")
            duracion = random.randint(t_min, t_max)
            yield self.env.timeout(duracion)
            self.registrar_evento("FINALIZADO")

### 4. Definimos la clase que representa una simulación del movimiento en el banco

In [161]:
class Simulacion:
    def __init__(self, env, num_clientes, num_atms, num_cajas, num_agentes):
        self.env = env
        self.banco = Banco(env, num_atms, num_cajas, num_agentes)
        self.clientes = []

        for i in range(num_clientes):
            llegada = random.expovariate(1 / 9) # genera un número aleatorio siguiendo una distribución exponencial con tasa media de 1/9
            env.process(self.lanzar_cliente(i, llegada))

    def lanzar_cliente(self, id, delay):
        yield self.env.timeout(delay)

        # Se crea un cliente y se añade a la lista de clientes
        cliente = Cliente(self.env, id, self.banco)
        self.clientes.append(cliente) # añadir cliente a la lista

### 5. Iniciar la simulación

In [162]:
# Creación del entorno de simulación
env = simpy.Environment()  # Instancia el entorno de simulación

random.seed(111)  # Fija la semilla aleatoria para reproducibilidad

# Configuración de parámetros de la simulación
num_clientes = 30    # Número total de clientes a simular
num_atms = 2         # Número de cajeros automáticos (ATMs) disponibles
num_cajas = 4        # Número de cajas tradicionales disponibles
num_agentes = 3      # Número de agentes de servicio al cliente

# Inicialización de la simulación
sim = Simulacion(
    env,              # Entorno de simulación
    num_clientes,     # Parámetros pasados al constructor
    num_atms,
    num_cajas,
    num_agentes
)

# Ejecución de la simulación
env.run(until=60 * 8)  # Corre la simulación por 8 horas (480 minutos)

### 6. Resultados de la simulación

Perfil de los clientes

In [163]:
# Construcción del dataframe del perfil del cliente
lista_clientes = []
for c in sim.clientes:
    lista_clientes.append({
        "id": c.id,
        "tipo_servicio": c.tipo_servicio,
        "operacion": c.operacion,
        "servicio_personalizado": c.servicio_personalizado
    })

dfclientes = pd.DataFrame(lista_clientes)
dfclientes

,id,tipo_servicio,operacion,servicio_personalizado
0,27,caja,Consulta,True
1,26,servicio_cliente,Problemas de cuenta,True
2,6,ATM,Consulta,False
3,7,ATM,Retiro,False
4,1,ATM,Consulta,False
5,29,caja,Deposito,True
6,14,servicio_cliente,Problemas de cuenta,True
7,9,ATM,Consulta,False
8,21,servicio_cliente,Problemas de cuenta,True
9,13,servicio_cliente,Problemas de cuenta,True


Detalle de los eventos 

In [164]:
lista_eventos = []
for c in sim.clientes:
    lista_eventos.append( c.eventos )

lista_eventos

dfeventos = pd.concat(lista_eventos)
dfeventos = dfeventos.reset_index()
dfeventos.drop( ["index"], axis=1, inplace=True )

dfeventos

,id,evento,time,servicio,operacion
0,27,LLEGÓ,3.941225,caja,Consulta
1,27,ATENDIDO,3.941225,caja,Consulta
2,27,FINALIZADO,9.941225,caja,Consulta
3,26,LLEGÓ,4.026502,servicio_cliente,Problemas de cuenta
4,26,ATENDIDO,4.026502,servicio_cliente,Problemas de cuenta
...,...,...,...,...,...
85,16,ATENDIDO,23.955940,ATM,Consulta
86,16,FINALIZADO,26.955940,ATM,Consulta
87,2,LLEGÓ,28.706777,ATM,Deposito
88,2,ATENDIDO,28.706777,ATM,Deposito


### Respuesta a preguntas

6.1 ¿ Cuál fue el tiempo promedio de tiempo de espera por tipo de servicio?

In [ ]:
# Calcular el tiempo de espera para cada cliente
dfeventos["lag_time"] = dfeventos.groupby("id")["time"].shift(1)
df_eventos_atendidos = dfeventos.loc[dfeventos["evento"] == "ATENDIDO"].copy()
df_eventos_atendidos["tiempo_espera"] = df_eventos_atendidos["time"] - df_eventos_atendidos["lag_time"]

# Calcular el promedio de tiempo de espera por tipo de servicio
tiempo_promedio_espera = df_eventos_atendidos.groupby("servicio")["tiempo_espera"].mean()
print(tiempo_promedio_espera)

servicio
ATM                 1.505602
caja                0.179106
servicio_cliente    4.582433
Name: tiempo_espera, dtype: float64


6.2 ¿Cuántos clientes fueron atendidos en total por tipo de servicio?

In [166]:
# Contar los clientes atendidos por tipo de servicio
clientes_atendidos = df_eventos_atendidos.groupby("servicio")["id"].nunique()
print(clientes_atendidos)

servicio
ATM                 18
caja                 7
servicio_cliente     5
Name: id, dtype: int64


6.3 ¿Cuál fue el tiempo total promedio que un cliente pasó en el banco? ( desde su llegada hasta su salida )

In [172]:
# Obtener el evento LLEGÓ y FINALIZADO para calcular el tiempo total en el banco
dfeventos["lag_time_finalizado"] = dfeventos.groupby("id")["time"].shift(1)  # Evento anterior a "FINALIZADO"
df_eventos_finalizados = dfeventos.loc[dfeventos["evento"] == "FINALIZADO"].copy()
df_eventos_finalizados["tiempo_total"] = df_eventos_finalizados["time"] - df_eventos_finalizados["lag_time_finalizado"]

# Calcular el promedio de tiempo total
tiempo_promedio_total = df_eventos_finalizados["tiempo_total"].mean()
print("Tiempo promedio total: ", tiempo_promedio_total)


Tiempo promedio total:  5.0


6.4 ¿Cuál fue el nivel de ocupación de cada recursos?

In [ ]:
def calcular_ocupacion(recurso, df_eventos):
    """
    Calcula el tiempo de ocupación promedio para un tipo de recurso específico.
    
    Args:
        recurso (str): Nombre del recurso a analizar ('ATM', 'caja', 'servicio_cliente')
        df_eventos (pd.DataFrame): DataFrame con los registros de eventos
        
    Returns:
        float: Tiempo promedio de ocupación del recurso
    """

    # Filtrar por el recurso
    df_recurso = df_eventos.loc[df_eventos["servicio"] == recurso].copy()
    
    # Validar que existan datos para el recurso
    if df_recurso.empty:
        print(f"Advertencia: No hay datos para el recurso '{recurso}'")
        return 0.0
    
    # Calcular la duración de cada uso de recurso
    df_recurso["tiempo_ocupado"] = (
        df_recurso.groupby("id")["time"].transform("max") - 
        df_recurso.groupby("id")["time"].transform("min")
    )
    
    # Calcular el nivel de ocupación promedio
    return df_recurso["tiempo_ocupado"].mean()

ocupacion_atm = calcular_ocupacion("ATM", dfeventos)
ocupacion_caja = calcular_ocupacion("caja", dfeventos)
ocupacion_agentes = calcular_ocupacion("servicio_cliente", dfeventos)

print(f"Ocupación promedio ATM: {ocupacion_atm:.2f} unidades de tiempo")
print(f"Ocupación promedio cajas: {ocupacion_caja:.2f} unidades de tiempo")
print(f"Ocupación promedio agentes: {ocupacion_agentes:.2f} unidades de tiempo")

Ocupación promedio ATM: 3.73 unidades de tiempo
Ocupación promedio cajas: 4.18 unidades de tiempo
Ocupación promedio agentes: 20.98 unidades de tiempo


6.5 ¿Cuál fue el tiempo máximo de espera registrado y en que servicio ocurre?

In [174]:
# Obtener fila con el tiempo máximo de espera
espera_max = df_eventos_atendidos.loc[df_eventos_atendidos["tiempo_espera"].idxmax()] # devuelve el índice (número de fila) donde se encuentra el valor máximo de la columna

# Mostrar el resultado
print(f"Tiempo máximo de espera: {espera_max['tiempo_espera']} unidades de tiempo.")
print(f"Ocurrió en el servicio: {espera_max['servicio']}")
print(f"ID del cliente: {espera_max['id']}")

Tiempo máximo de espera: 13.09809237530427 unidades de tiempo.
Ocurrió en el servicio: servicio_cliente
ID del cliente: 13
